Import Stuff

In [1]:
import ssl
import re

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from hdbscan import HDBSCAN
from umap import UMAP

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split


import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from datetime import datetime
from textblob import TextBlob

import pickle
import requests

import importlib
import functions

import torch
import datasets
from datasets import load_dataset
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments





In [ ]:
from functions import *
importlib.reload(functions)

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

parties = ["CDU", "SPD", "FDP", "AFD", "LEFT", "GREENS"]
file_path = "C:/Users/Jacob/OneDrive/uni/MA WiSoz/Semester III/Computational Social Sciences/foPra/data/"


# Open the file in read mode
with open("API Keys/DeepL.txt", 'r', encoding='utf-8') as file:
    # Read the contents of the file
    api_key_deepl = file.read()

url = 'https://api-free.deepl.com/v2/translate'

# Load the DataFrame from a pickle file
df_manifesto = pd.read_pickle('data\df_manifesto.pkl') 
coding_list = pd.read_pickle('data/coding_list.pkl')
topics = pd.read_pickle('topics.pkl') 


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Edit Text documents

In [3]:
lemmatizer = WordNetLemmatizer()

defining different models
    

In [4]:
embedding_model_en = "all-MiniLM-L6-v2"
embedding_model_multilingual1 = "paraphrase-multilingual-mpnet-base-v2"
embedding_model_multilingual2 = "distiluse-base-multilingual-cased-v1"
embedding_model_manifestoberta = "manifesto-project/manifestoberta-xlm-roberta-56policy-topics-sentence-2024-1-1"

In [5]:
representation_model = KeyBERTInspired()
cluster_model = KMeans(n_clusters=30)

In [7]:
umap_model = UMAP(
    n_neighbors=30, 
    n_components=5, 
    min_dist=0.1, 
    metric='cosine', 
    random_state=42
    )

In [6]:
hdbscan_model = HDBSCAN(
    min_cluster_size=20, 
    # min_samples=10,
    metric='euclidean', 
    prediction_data=True
    )



In [8]:
seed_topic_list = [
    ["steuergeld", 
    "ausgaben", 
    "staatsausgaben", 
    "staatshaushalt", 
    "schulden", 
    "schuldenbremse", 
    "staatsschulden", 
    "defizit", 
    "haushalt", 
    "haushaltsdefizit", 
    "haushaltsausgleich", 
    "schwarze null", 
    "schuldenabbau", 
    "schuldenpolitik", 
    "schuldenkrise", 
    "schuldenlast", 
    "schuldenstand", 
    "schuldenquote", 
    "schuldenpolitisch",
    "ausgaben erhöhen",
    "ausgaben kürzen"]]

Initialisiere und trainiere das BERTopic-Modell


In [9]:

# Ensure the 'text' column contains only strings and handle NaN values
df_manifesto["text"] = df_manifesto["text"].astype(str).fillna("")

topic_model = BERTopic(
    language="multilingual"
    , n_gram_range=(1,2)
    , min_topic_size=20
    # , top_n_words=20
    , representation_model=representation_model
    , embedding_model=embedding_model_multilingual2
    , umap_model=umap_model
    , hdbscan_model=hdbscan_model
    # , seed_topic_list=seed_topic_list
    )

#topic_model = BERTopic(nr_topics=100, calculate_probabilities=True, embedding_model="all_MiniLM-L6-v2")
topics, probs = topic_model.fit_transform(df_manifesto["text"])



In [16]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,48000,-1_wollen wir_wir wollen_wir werden_werden wir,"[wollen wir, wir wollen, wir werden, werden wi...",[Wir werden uns an keiner Regierung beteiligen...
1,0,4872,0_werden wir_familie und_familien_für kinder,"[werden wir, familie und, familien, für kinder...",[Mit Blick auf die politisch gewünschten Ziele...
2,1,1911,1_steuerreform_einkommensteuer_besteuert werde...,"[steuerreform, einkommensteuer, besteuert werd...",[BÜNDNIS 90DIE GRÜNEN wollen den radikalen Abb...
3,2,1887,2_der patientinnen_krankenhäusern_medizinische...,"[der patientinnen, krankenhäusern, medizinisch...",[Unser Ziel ist die Rechte der Patientinnen un...
4,3,1727,3_deutschland ist_deutsche staatsangehörigkeit...,"[deutschland ist, deutsche staatsangehörigkeit...","[Deutschland ist ein Einwanderungsland, Deutsc..."
...,...,...,...,...,...
184,183,21,183_sanktionen abschaffen_sanktionen und_sankt...,"[sanktionen abschaffen, sanktionen und, sankti...",[Sperrzeiten und Sanktionen werden ausnahmslos...
185,184,21,184_gute erwerbsarbeit_erwerbstätigkeit gute_g...,"[gute erwerbsarbeit, erwerbstätigkeit gute, gu...",[Gute Arbeit bedeutet dass wir von unserer Erw...
186,185,21,185_flughäfen verbessern_überflüssige regional...,"[flughäfen verbessern, überflüssige regionalfl...",[Die Rechte von Flugpassagieren haben wir verb...
187,186,21,186_2030 wollen_2020 wollen_erneuerbare energi...,"[2030 wollen, 2020 wollen, erneuerbare energie...",[Bis zum Jahr 2020 wollen wir den Anteil der E...


In [ ]:
df_manifesto["topic"] = topics
topic_list =[57, 62, 65, 145, 169, ]

In [22]:
df_manifesto.rename(columns={"label": "category"}, inplace=True)

In [63]:
# Filter only relevant codes
df_spendings = df_manifesto[df_manifesto["cmp_code"].isin([
    303,
    401,
    411, 
    409, 
    414,
    103.2,
    # 402,
    504,
    505,
    506,
    507,
]) | df_manifesto["topic"].isin(topic_list)]

# Map policy codes to labels
label_map = {
    411: 0, 
    409: 0,
    401: 1,
    303: 1,
    414: 1,
    103.2: 0,
    504: 0,
    505: 1,
    506: 0,
    507: 1
    }  # 0 = expansion, 1 = austerity
    
df_spendings["label"] = df_spendings["cmp_code"].map(label_map)
# df_spendings["label"] = df_spendings["cmp_code"].map(label_map)

In [64]:
df_no_label = df_spendings[df_spendings.isna().any(axis=1)]
df_no_label = df_no_label[[col for col in df_no_label.columns if col != 'label']]
df_no_label.insert(2, 'label', df_spendings['label'])
df_no_label.drop(columns=['description_md'], inplace=True)
df_no_label["label"] = coding_list["label"]

In [68]:
df_spendings["label"].update(df_no_label["label"])

df_spendings.dropna(subset=["label"], inplace=True)
df_spendings["label"] = df_spendings["label"].astype(int)

In [69]:


df_manifesto.to_pickle('data/df_manifesto_final.pkl')
df_spendings.to_pickle('data/df_spendings_final.pkl')
# df_no_label.to_csv('data/Unlabeled.csv', sep=';', index=True, encoding='utf-8-sig')

In [37]:
len(df_no_label)

1485

In [ ]:
topic_model.get_topic(83)

In [86]:
topic_model.get_topic(80)
related_terms = topic_model.find_topics("related", top_n=10)
print(related_terms)

([83, 131, 156, 128, 102, 160, 152, 94, 142, 96], [0.18113098, 0.13463318, 0.10732683, 0.08718705, 0.086810865, 0.076846965, 0.07013552, 0.059989497, 0.03488005, 0.031079182])


In [ ]:
len(topic_model.get_topic(25))

In [ ]:

# Erstelle das `topics_over_time`-Objekt
topics_over_time = topic_model.topics_over_time(df_manifesto["text"], df_manifesto["date"])
#topic_model = BERTopic(representation_model=KeyBERTInspired())


In [ ]:
# Save topics_over_time to a file
with open("topics_over_time.pkl", "wb") as f:
    pickle.dump(topics_over_time, f)

In [14]:

# Method 1 - safetensors
topic_model.save(file_path, serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model_manifestoberta)

# Method 2 - pytorch
#topic_model.save(file_path, serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# Method 3 - pickle
#topic_model.save("bertopic_model.pkl")



In [ ]:
predicted_class = model.config.id2label[logits.argmax().item()]
print(predicted_class)
# 201 - Freedom and Human Rights


In [14]:
predicted_class = model.config.id2label[logits.argmax().item()]
print(predicted_class)
# 501 - Environmental Protection: Positive

414 - Economic Orthodoxy


In [15]:
# Save the topics to a pickle file
with open('topics.pkl', 'wb') as f:
	pickle.dump(topics, f)

# Save the probabilities to a pickle file
with open('probs.pkl', 'wb') as f:
	pickle.dump(probs, f)